In [41]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('/datasets/stop_words_en.txt') as f:
    stop_words = set(f.read().split())

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word in stop_words or len(word)==1:
            print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        else:
            print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
            print "%s\t%s\t%d" % (''.join(sorted(word.lower())), word.lower(), 1)

Overwriting mapper.py


In [42]:
%%writefile reducer.py


import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

current_key = None
current_cnt = 0
words_set = set()

for line in sys.stdin:
    try:
        key, cnt, word = unicode(line.strip()).split('\t')
        cnt = int(cnt)
    except ValueError as e:
        continue
    
    if current_key != key:
        if current_key and (len(words_set) > 1):
            print "%d\t%d\t%s" % (current_cnt, len(words_set), ','.join(sorted(words_set)))
        current_key = key
        words_set = set()
        words_set.add(word)
        current_cnt = cnt
    else:
        words_set.add(word)
        current_cnt += cnt
        
print "%d\t%d\t%s" % (current_cnt, len(words_set), ','.join(sorted(words_set)))

Overwriting reducer.py


In [47]:
%%bash

OUT_DIR="word_cn"
NUM_REDUCERS=4

# echo ${OUT_DIR}

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat word_cn/* | grep -P '(,|\t)english($|,)'


18/04/27 02:04:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/27 02:04:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/27 02:04:26 INFO mapred.FileInputFormat: Total input files to process : 1
18/04/27 02:04:26 INFO mapreduce.JobSubmitter: number of splits:2
18/04/27 02:04:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1524702992247_0013
18/04/27 02:04:27 INFO impl.YarnClientImpl: Submitted application application_1524702992247_0013
18/04/27 02:04:27 INFO mapreduce.Job: The url to track the job: http://4982f6c0e4f0:8088/proxy/application_1524702992247_0013/
18/04/27 02:04:27 INFO mapreduce.Job: Running job: job_1524702992247_0013
18/04/27 02:04:33 INFO mapreduce.Job: Job job_1524702992247_0013 running in uber mode : false
18/04/27 02:04:33 INFO mapreduce.Job:  map 0% reduce 0%
18/04/27 02:04:49 INFO mapreduce.Job:  map 20% reduce 0%
18/04/27 02:04:55 INFO mapreduce.Job:  map 31% reduce 0%
18/04/27 02:05:01 INFO 